In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [52]:
# This is a sequence of length 9
seq_single = torch.tensor([[
    [1, 2, 1],  # <-- Head of seq
    [3, 2, 1],  # <-- Each step with input dim being 3
    [4, 4, 3],
    [0, 2, 1],
    [3, 1, 1],
    [5, 2, 1],
    [1, 9, 1],
    [0, 0, 1]   # <-- Tail of seq
]], dtype=torch.float)

# Thus, this is a batch with only ONE sample
#   Batch_Size x Seq_Length x Else (e.g., dim of word vector)
seq_single.size()

torch.Size([1, 8, 3])

In [70]:
# Initialize a basic rnn model
rnn_single = nn.RNN(
    input_size=3        # <-- 3D input, e.g. each word vec is of dim 3
    , hidden_size=1     # <-- Compress the input to a scalar
    , num_layers=3      # <-- Obviously, 3 hidden layers
    , batch_first=True  # <-- Default to false; in our sample, the first data dimension is the batch dim
)

# Feed in our batch
out_single, hid_single = rnn_single(seq_single)

In [71]:
# This would be the output of the last hidden layer at EACH STEP
out_single

tensor([[[0.0486],
         [0.0763],
         [0.0752],
         [0.1037],
         [0.1004],
         [0.1343],
         [0.1768],
         [0.1569]]], grad_fn=<TransposeBackward1>)

In [72]:
# The out put of EACH of the three hidden layers at the LAST STEP
#   Note how the output of last element corresponds to that of <out_single>
hid_single

tensor([[[-0.8551]],

        [[-0.9824]],

        [[ 0.1569]]], grad_fn=<StackBackward>)